# Apache Beam - Create

The `Create` class takes as input a collection of Java elements of a given type (T) and returns a `PCollection` containing those elements.  i.e.  `PCollection<T>`


* [JavaDoc: Class Create](https://beam.apache.org/releases/javadoc/2.42.0/org/apache/beam/sdk/transforms/Create.html)


First, we define the dependencies that we wish to load from the Maven repositories.

In [1]:
%%loadFromPOM

<dependency>
  <groupId>org.apache.beam</groupId>
  <artifactId>beam-sdks-java-core</artifactId>
  <version>2.43.0</version>
</dependency>

<dependency>
  <groupId>org.apache.beam</groupId>
  <artifactId>beam-runners-direct-java</artifactId>
  <version>2.43.0</version>
  <scope>runtime</scope>
</dependency>

<dependency>
    <groupId>org.slf4j</groupId>
    <artifactId>slf4j-api</artifactId>
    <version>2.0.6</version>
</dependency>

Next we define our imports required for execution.

In [2]:
import java.util.Arrays;
import java.util.List;

import org.apache.beam.sdk.Pipeline;
import org.apache.beam.sdk.options.Default;
import org.apache.beam.sdk.options.Description;
import org.apache.beam.sdk.options.PipelineOptionsFactory;
import org.apache.beam.sdk.options.PipelineOptions;
import org.apache.beam.sdk.options.StreamingOptions;
import org.apache.beam.sdk.transforms.Create;
import org.apache.beam.sdk.values.PCollection;
import org.apache.beam.sdk.transforms.DoFn;
import org.apache.beam.sdk.transforms.ParDo;
import org.apache.beam.sdk.coders.KvCoder;
import org.apache.beam.sdk.coders.StringUtf8Coder;
import org.apache.beam.sdk.values.KV;
import org.apache.beam.sdk.schemas.annotations.DefaultSchema;
import org.apache.beam.sdk.schemas.JavaBeanSchema;

String args[] = new String[] {};
var options = PipelineOptionsFactory.fromArgs(args).withValidation().create();

First, here is a PCollection of strings:

In [3]:
public class LoggingDoFn<T> extends DoFn<T, T>  {
  @ProcessElement
  public void processElement(@Element T element, OutputReceiver<T> out) {
    System.out.println(element);
    out.output(element);
  }
}

var pipeline = Pipeline.create(options);
pipeline
  .apply("Create elements", Create.of(Arrays.asList("Hello!", "World!")))
  .apply("Print elements",ParDo.of(new LoggingDoFn<>()));
pipeline.run().waitUntilFinish();

World!
Hello!


DONE

And now we create a PCollection of numbers:

In [4]:
var pipeline = Pipeline.create(options);
pipeline
  .apply("Create elements", Create.of(3,4,5))
  .apply("Print elements",ParDo.of(new LoggingDoFn<>()));
pipeline.run().waitUntilFinish();

3
5
4


DONE

And now some key/value pairs:

In [5]:
 Set<KV<String, String>> set = new HashSet<KV<String, String>>();
 set.add(KV.of("red", "John"));
 set.add(KV.of("blue", "Sarah"));
 set.add(KV.of("blue", "Mike"));
 set.add(KV.of("red", "Fred"));
 set.add(KV.of("green", "Joan"));

var pipeline = Pipeline.create(options);
pipeline
  .apply("Create elements", Create.of(set))
  .apply("Print elements",ParDo.of(new LoggingDoFn<>()));
pipeline.run().waitUntilFinish();


KV{blue, Sarah}
KV{red, John}
KV{red, Fred}
KV{blue, Mike}
KV{green, Joan}


DONE

And now with some classes.  In this example, we annotate with a default generated schema.

In [6]:
@DefaultSchema(JavaBeanSchema.class)
class Person {
  private String name;
  private int age;
  public Person() {}
  public Person(String name, int age) {
    this.name = name;
    this.age = age;
  }
  public void setName(String name) {
    this.name = name;
  }
  public String getName() {
    return name;
  }
  public void setAge(int age) {
    this.age = age;
  }
  public int getAge() {
    return age;
  }
  public String toString() {
    return "Name: " + name + ", age: " + age;
  }
}

var pipeline = Pipeline.create(options);
pipeline
  .apply("Create elements", Create.of(new Person("Neil", 41), new Person("Sue", 27)))
  .apply("Print elements",ParDo.of(new LoggingDoFn<>()));
pipeline.run().waitUntilFinish();

Name: Sue, age: 27
Name: Neil, age: 41


DONE

But what if we can't create a schema?  What if there is no coder for the class?

In [9]:
class Person2 {
  private String name;
  private int age;

  public Person2(String name, int age) {
    this.name = name;
    this.age = age;
  }

  public String toString() {
    return "Name: " + name + ", age: " + age;
  }
}

var pipeline = Pipeline.create(options);
pipeline
  .apply("Create elements", Create.of(new Person2("Neil", 41), new Person2("Sue", 27)))
  .apply("Print elements",ParDo.of(new LoggingDoFn<>()));
pipeline.run().waitUntilFinish();

EvalException: Unable to infer a coder and no Coder was specified. Please set a coder by invoking Create.withCoder() explicitly  or a schema by invoking Create.withSchema().

The easiest solution is to ensure the class is serializable:

In [10]:
class Person2 implements Serializable{
  private String name;
  private int age;

  public Person2(String name, int age) {
    this.name = name;
    this.age = age;
  }

  public String toString() {
    return "Name: " + name + ", age: " + age;
  }
}

var pipeline = Pipeline.create(options);
pipeline
  .apply("Create elements", Create.of(new Person2("Neil", 41), new Person2("Sue", 27)))
  .apply("Print elements",ParDo.of(new LoggingDoFn<>()));
pipeline.run().waitUntilFinish();

Name: Sue, age: 27
Name: Neil, age: 41


DONE